# $Ti_2AlC$ DFT计算验证

## 1. Convergence Test

### 1.1 ecutwfc convergence test (only valid for pw basis)

In [ ]:
from convergence_test import ABACUSConvergenceTest

conv_test = ABACUSConvergenceTest()
conv_test.ecut_run(ecut_min=50, ecut_max=130, ecut_interval=10)
conv_test.ecut_postprocessing()
conv_test.ecut_generate_plot()

### 1.2 kspacing convergence test

In [ ]:
from convergence_test import ABACUSConvergenceTest

conv_test = ABACUSConvergenceTest()
conv_test.kpoint_run(kspacing_min=0.05, kspacing_max=0.15, kspacing_interval=0.01)
conv_test.kpoint_postprocessing()
conv_test.kpoint_generate_plot()

### 1.3 kspacing convergence test (x, y, z)

In [ ]:
from convergence_test import ABACUSConvergenceTest

conv_test = ABACUSConvergenceTest()
conv_test.kpoint_xyz_run(kspacing_min_xyz=[0.05, 0.05, 0.05], kspacing_max_xyz=[0.15, 0.15, 0.15], kspacing_interval_xyz=0.02)
conv_test.kpoint_xyz_postprocessing()
conv_test.kpoint_xyz_generate_plot()

## 2. Cell-relax

In [ ]:
# install ase-abacus first
! pip install git+https://gitlab.com/1041176461/ase-abacus.git

- 使用 ASE 计算初始（原）晶格常数以及体积

In [ ]:
from ase.io import read
from pathlib import Path

cs_dir = "cell-relax"

cs_stru_original = Path(cs_dir, "STRU")
cs_atoms_original = read(cs_stru_original, format="abacus")

lattice_params_original = cs_atoms_original.cell.cellpar()
volume_original = cs_atoms_original.cell.volume

# format: [len(a), len(b), len(c), angle(b,c), angle(a,c), angle(a,b)]
print("Original lattice_params: ", lattice_params_original)
print("Original volume: ", volume_original)

In [ ]:
from ase.io import read
from pathlib import Path
from ase.visualize import view

cs_dir = "cell-relax"

cs_stru_original = Path(cs_dir, "STRU")
cs_atoms_original = read(cs_stru_original, format="abacus")
view(cs_atoms_original, viewer='ngl')

### 2.1 PW

- 使用如下命令运行 Abacus 进行松弛

In [ ]:
! cd cell-relax && cp INPUT_pw INPUT && abacus

- 使用 ASE 计算晶格常数以及晶胞体积

In [2]:
from ase.io import read
from pathlib import Path

cs_dir = "cell-relax"

cs_stru_relaxed = Path(cs_dir, "OUT.ABACUS", "STRU_ION_D")
cs_atoms_relaxed = read(cs_stru_relaxed, format="abacus")

lattice_params_relaxed = cs_atoms_relaxed.cell.cellpar()
volume_relaxed = cs_atoms_relaxed.cell.volume

# format: [len(a), len(b), len(c), angle(b,c), angle(a,c), angle(a,b)]
print("Relaxed lattice_params: ", lattice_params_relaxed)
print("Relaxed volume: ", volume_relaxed)

Relaxed lattice_params:  [  3.0669413    3.0669413   13.73942335  90.          90.
 120.        ]
Relaxed volume:  111.92060896338533


### 2.2 LCAO

- 使用如下命令运行 Abacus 进行松弛

In [ ]:
! cd cell-relax && cp INPUT_lcao INPUT && abacus

- 使用 ASE 计算晶格常数以及晶胞体积

In [ ]:
from ase.io import read
from pathlib import Path

cs_dir = "cell-relax"

cs_stru_relaxed = Path(cs_dir, "OUT.ABACUS", "STRU_ION_D")
cs_atoms_relaxed = read(cs_stru_relaxed, format="abacus")

lattice_params_relaxed = cs_atoms_relaxed.cell.cellpar()
volume_relaxed = cs_atoms_relaxed.cell.volume

# format: [len(a), len(b), len(c), angle(b,c), angle(a,c), angle(a,b)]
print("Relaxed lattice_params: ", lattice_params_relaxed)
print("Relaxed volume: ", volume_relaxed)

- 理论计算与实验对比

[1] B. Manoun, F. X. Zhang, S. K. Saxena, T. El-Raghy, and M. W. Barsoum, “X-ray high-pressure study of Ti2AlN and Ti2AlC,” Journal of Physics and Chemistry of Solids, vol. 67, no. 9–10, pp. 2091–2094, Sep. 2006, doi: 10.1016/j.jpcs.2006.05.051.

| 参数 | 文献实验值 | 计算结果 (LCAO) | 相对误差 (LCAO) | 计算结果 (PW) | 相对误差 (PW) |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **晶格常数 a (Å)** | 3.065 ± 0.004 | 3.068 | +0.10% | 3.067 | +0.07% |
| **晶格常数 c (Å)** | 13.71 ± 0.03 | 13.750 | +0.29% | 13.739 | +0.21% |
| **晶胞体积 V₀ (Å³)** | 111.6 ± 0.6 | 112.07 | +0.42% | 111.92 | +0.29% |

## 3. Elastic Constants

### 3.1 Introduction

#### 3.1.1. 核心概念：什么是弹性常数？

**弹性 (Elasticity)** 是固体材料在外力作用下发生形变，当外力撤去后能恢复其原始形状和尺寸的物理性质。

在线弹性范围内，材料的 **应力 (Stress, $\sigma$)** 与 **应变 (Strain, $\epsilon$)** 之间遵循胡克定律 (Hooke's Law)，即两者成正比线性关系。

* **应力 $\sigma$**：描述材料内部单位面积上所受的力，反映了内力的强度。单位通常是帕斯卡 (Pa) 或吉帕斯卡 (GPa)。
* **应变 $\epsilon$**：描述材料的相对形变程度，是一个无量纲的量（或表示为百分比）。

**弹性常数 (Elastic Constant, $C$)** 正是这个线性关系中的比例系数，它**定量地衡量了材料抵抗弹性形变的能力**。其物理意义可以理解为材料的“刚度”或“硬度”。

> **一句话总结：弹性常数越大，使材料发生单位形变所需要的力就越大，材料也就越“硬”。**

#### 3.1.2. 弹性常数的表示

##### 第一步：理解“阶” (Rank) - 从简单到复杂

我们可以把“张量”看作是描述物理量的一种数学工具，它的“阶”决定了它的复杂程度。

* **零阶张量 (Scalar)**：就是一个**单独的数**，没有方向。
    * **例子**：温度、质量。比如房间温度是 25°C，一个数字就说清楚了。

* **一阶张量 (Vector)**：就是一个**矢量**，既有大小又有方向。在三维空间中，我们需要用 **3 个数**来完整描述它。
    * **例子**：力、速度。比如一个力 $\vec{F}$，我们可以分解为它在 x, y, z 三个方向上的分量 $(F_x, F_y, F_z)$。它有一个下标，所以是“一阶”。

* **二阶张量 (Matrix)**：你可以把它想象成一个**矩阵**。它描述的是一个方向上的输入和另一个方向上的输出之间的关系。在三维空间中，它是一个 **3x3 的矩阵**，所以有 $3 \times 3 = 9$ 个分量。
    * **例子**：应力 ($\sigma$) 和应变 ($\epsilon$)。

##### 第二步：为什么应力($\sigma$)和应变($\epsilon$)是“二阶”的？

让我们以“应力”为例，它比“力”要复杂。

想象在材料内部有一个极小的正方体。

1.  我们先关注一个面，比如**垂直于 x 轴的面**（我们称之为 x-面）。
2.  作用在这个 x-面上的力，本身就是一个矢量，它可以有三个方向的分量：
    * 一个**垂直**于该面的力 (正应力)，方向在 x 轴上，记为 $\sigma_{xx}$。
    * 两个**平行**于该面的力 (剪应力)，方向分别在 y 和 z 轴上，记为 $\sigma_{xy}$ 和 $\sigma_{xz}$。

    你看，为了描述**一个面**上的受力情况，我们就需要 3 个数。

3.  现在，这个小方块有 3 个主要的面（x-面, y-面, z-面）。每个面都需要 3 个数来描述其受力。
4.  所以，总共就需要 $3 \times 3 = 9$ 个数，才能完整描述这个点上各个方向的受力状态。

这 9 个数可以排列成一个 3x3 的矩阵：
$$
\boldsymbol{\sigma} = \begin{pmatrix} \sigma_{xx} & \sigma_{xy} & \sigma_{xz} \\ \sigma_{yx} & \sigma_{yy} & \sigma_{yz} \\ \sigma_{zx} & \sigma_{zy} & \sigma_{zz} \end{pmatrix}
$$

这个矩阵就是**二阶应力张量**。它有两个下标（$ij$ in $\sigma_{ij}$），第一个下标 $i$ 代表“哪个面”，第二个下标 $j$ 代表“力的方向”。应变张量 $\boldsymbol{\epsilon}$ 也是完全一样的道理。

> **小结**：应力和应变之所以是二阶张量（9个分量），是因为它们需要同时描述“作用面”和“作用力/形变”两个方向的信息。

##### **第三步：为什么弹性常数($C_{ijkl}$)是“四阶”的？**

现在到了最关键的一步。我们知道胡克定律是 `应力 = 弹性常数 × 应变`。

我们已经清楚了：
* **应力** $\sigma_{ij}$ 是一个有 9 个分量的矩阵。
* **应变** $\epsilon_{kl}$ 也是一个有 9 个分量的矩阵。

**弹性常数 $C$ 的作用，就是建立这两个矩阵之间每一个分量的联系**。它是一个“转换器”，输入一个应变矩阵，输出一个应力矩阵。

让我们想一下这个“转换器”需要多少个零件：

1.  我们想求出应力矩阵中的**一个分量**，比如 $\sigma_{xx}$。
2.  $\sigma_{xx}$ 的大小，可能**同时受到所有 9 种应变分量** ($\epsilon_{xx}, \epsilon_{xy}, \epsilon_{xz}, \epsilon_{yx}, \dots, \epsilon_{zz}$) 的影响。
3.  所以，为了计算 $\sigma_{xx}$，我们需要 9 个系数，分别对应它和 9 个应变分量的关系：
    $$
    \sigma_{xx} = C_{xxxx}\epsilon_{xx} + C_{xxxy}\epsilon_{xy} + C_{xxxz}\epsilon_{xz} + \dots (\text{共9项})
    $$

4.  仅仅为了得到应力矩阵的**一个**分量 $\sigma_{xx}$，我们就需要 9 个弹性常数分量 ($C_{xxkl}$)。

5.  而应力矩阵本身有 9 个分量 ($\sigma_{xx}, \sigma_{xy}, \dots, \sigma_{zz}$)。

6.  因此，要描述所有应力分量和所有应变分量之间的完整关系，总共需要的零件（弹性常数的分量）数量就是：
    $$
    (\text{应力分量的数量}) \times (\text{应变分量的数量}) = 9 \times 9 = 81 \text{ 个}
    $$

这个需要 81 个分量来描述的“超级转换器”，就是一个**四阶张量**。它有四个下标 $C_{ijkl}$，因为每个下标都可以在 x, y, z 三个方向中取值，所以总数是 $3 \times 3 \times 3 \times 3 = 3^4 = 81$。


#### 3.1.3. 数学描述：从张量到 Voigt 矩阵

在三维空间中，应力和应变都是二阶对称张量，有9个分量。因此，描述它们之间完整关系的弹性常数是一个复杂的四阶张量 $C_{ijkl}$，包含 $3 \times 3 \times 3 \times 3 = 81$ 个分量。

$$\sigma_{ij} = \sum_{k,l} C_{ijkl} \epsilon_{kl} \quad (i, j, k, l = x, y, z)$$

幸运的是，由于应力张量 ($\sigma_{ij} = \sigma_{ji}$) 和应变张量 ($\epsilon_{kl} = \epsilon_{lk}$) 的对称性，以及能量守恒所要求的 $C_{ijkl} = C_{klij}$，独立的弹性常数分量远少于81个。

为了简化表示，我们引入 **Voigt 表示法 (Voigt Notation)**，将二阶张量“压缩”成一个6分量的矢量。

* **Voigt 映射规则**:
    * 拉伸/压缩分量: $xx \to 1$, $yy \to 2$, $zz \to 3$
    * 剪切分量: $yz \to 4$, $xz \to 5$, $xy \to 6$

* **应力与应变矢量**:
    * $\sigma = [\sigma_1, \sigma_2, \sigma_3, \sigma_4, \sigma_5, \sigma_6]^T$
    * $\epsilon = [\epsilon_1, \epsilon_2, \epsilon_3, \epsilon_4, \epsilon_5, \epsilon_6]^T$

经过 Voigt 变换后，四阶弹性张量 $C_{ijkl}$ 就简化为了一个 **6x6 的对称矩阵 $C_{ij}$**，胡克定律也相应写为更直观的矩阵形式：

$$
\begin{bmatrix} \sigma_1 \\ \sigma_2 \\ \sigma_3 \\ \sigma_4 \\ \sigma_5 \\ \sigma_6 \end{bmatrix}
=
\begin{bmatrix}
 C_{11} & C_{12} & C_{13} & C_{14} & C_{15} & C_{16} \\
 C_{12} & C_{22} & C_{23} & C_{24} & C_{25} & C_{26} \\
 C_{13} & C_{23} & C_{33} & C_{34} & C_{35} & C_{36} \\
 C_{14} & C_{24} & C_{34} & C_{44} & C_{45} & C_{46} \\
 C_{15} & C_{25} & C_{35} & C_{45} & C_{55} & C_{56} \\
 C_{16} & C_{26} & C_{36} & C_{46} & C_{56} & C_{66}
\end{bmatrix}
\begin{bmatrix} \epsilon_1 \\ \epsilon_2 \\ \epsilon_3 \\ 2\epsilon_4 \\ 2\epsilon_5 \\ 2\epsilon_6 \end{bmatrix}
$$

**注意**: 在工程定义中，剪切应变分量前有一个因子 2，这是为了保持能量表达式形式的一致性。

晶体的对称性会进一步减少独立弹性常数的数量。例如，对于立方晶体，只有 $C_{11}$, $C_{12}$, $C_{44}$ 三个是独立的。


#### 3.1.4. Abacus 计算流程：理论推导与实践步骤

##### **核心理论：求解线性方程组**

计算弹性常数的本质，是求解胡克定律的矩阵形式 $\boldsymbol{\sigma} = \mathbf{C} \boldsymbol{\varepsilon}$。在这个方程中：
* 应变矩阵 $\boldsymbol{\varepsilon}$ 是我们施加的**已知输入**。
* 应力矩阵 $\boldsymbol{\sigma}$ 是我们通过 DFT 计算得到的**测量结果**。
* 6x6 的弹性常数矩阵 $\mathbf{C}$ 是我们要求解的**未知量**。

我们的整个计算流程，就是一套设计精良的数值实验，旨在生成足够的线性方程组来精确地解出矩阵 $\mathbf{C}$ 的所有独立分量。

##### **步骤 1：获得理想的基态结构 (理论推导与结构弛豫)**

* **理论推导**
    胡克定律描述的是**应力变化量**与应变的关系，其更严谨的形式是 $\Delta\boldsymbol{\sigma} = \mathbf{C} \boldsymbol{\varepsilon}$，其中 $\Delta\boldsymbol{\sigma} = \boldsymbol{\sigma}_{\text{strained}} - \boldsymbol{\sigma}_0$。$\boldsymbol{\sigma}_0$ 是初始平衡态的参考应力。

    为了能将方程简化为 $\boldsymbol{\sigma} \approx \mathbf{C} \boldsymbol{\varepsilon}$，我们必须确保参考应力 $\boldsymbol{\sigma}_0$ 无限趋近于零。任何显著的残余应力 $\boldsymbol{\sigma}_0$ 都会成为整个计算的系统误差来源。

    在 DFT 中，一个体系的总能量 $E$ 是其原子坐标 $\{R_i\}$ 和晶格矢量 $\mathbf{A}$ 的函数。应力张量的定义是总能量对微小应变的偏导数：
    $$ \sigma_{ij} = \frac{1}{V} \frac{\partial E(\boldsymbol{\varepsilon}, \{R_i\})}{\partial \epsilon_{ij}} $$
    因此，获得零应力状态，等价于寻找一个能量对所有应变分量的偏导数都为零的结构，即找到体系的能量最低点。

* **实践步骤**
    1.  运行一次完整的几何优化（`calculation = 'cell-relax'`）。
    2.  此过程会同时调整晶胞参数和原子坐标，以寻找体系总能量的最小值，最终得到一个理论上应力为零的基态结构。
    3.  该结构的几何信息被保存在 `STRU_ION_D` 文件中，作为后续所有计算的“零点”参考。同时，需要记录下此时微小的残余应力值，用于最终结果的修正。

##### **步骤 2：施加受控应变 (理论推导与构型生成)**

* **理论推导**
    为了求解 $\mathbf{C}$ 矩阵，我们需要通过施加特定的应变来解耦方程。策略是**一次只激活一个应变分量**，从而孤立出 $\mathbf{C}$ 矩阵的一整列。

    以施加第一种应变模式为例，我们让 $\epsilon_1 \neq 0$，而所有其他的 $\epsilon_i (i=2 \dots 6) = 0$。此时，矩阵方程 $\boldsymbol{\sigma} = \mathbf{C} \boldsymbol{\varepsilon}$ 展开后变为：
    $$
    \begin{bmatrix} \sigma_1 \\ \sigma_2 \\ \sigma_3 \\ \sigma_4 \\ \sigma_5 \\ \sigma_6 \end{bmatrix}
    =
    \begin{bmatrix}
    C_{11} & C_{12} & \dots & C_{16} \\
    C_{21} & C_{22} & \dots & C_{26} \\
    \vdots & \vdots & \ddots & \vdots \\
    C_{61} & C_{62} & \dots & C_{66}
    \end{bmatrix}
    \begin{bmatrix} \epsilon_1 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \end{bmatrix}
    $$
    进行矩阵乘法后，我们得到一组非常简单的线性关系：
    $$
    \sigma_1 = C_{11}\epsilon_1 \quad \Rightarrow \quad C_{11} = \sigma_1 / \epsilon_1 \\
    \sigma_2 = C_{21}\epsilon_1 \quad \Rightarrow \quad C_{21} = \sigma_2 / \epsilon_1 \\
    \vdots \\
    \sigma_6 = C_{61}\epsilon_1 \quad \Rightarrow \quad C_{61} = \sigma_6 / \epsilon_1
    $$
    这清晰地表明：通过施加一个纯 $\epsilon_1$ 应变并测量由此产生的整个应力张量 $\boldsymbol{\sigma}$，我们就可以直接确定弹性矩阵 $\mathbf{C}$ 的**第一列**所有元素。

    同理，通过依次施加 6 种独立的应变模式，我们便能确定 $\mathbf{C}$ 矩阵的全部 6 列。而对每种模式施加多个不同的应变大小（例如 ±0.005, ±0.01），是为了通过多次测量进行线性拟合，以消除数值噪音，并验证计算是否在我们假设的线性弹性区内。

* **实践步骤**
    运行脚本将上述理论过程自动化，对基态结构 `STRU_ION_D` 施加 6 种模式 × 4 种大小的应变，生成 24 个包含变形后结构 (`STRU`) 的任务文件夹。

##### **步骤 3：计算应力响应 (理论推导与应力计算)**

* **理论推导**
    当我们对晶胞施加一个应变（即固定了晶格矢量）时，内部的原子会受到力的作用，不再处于平衡位置。此时计算出的瞬时应力并不代表材料在该应变下的真实响应。

    物理上，材料的应力响应应该是在**给定宏观应变下，内部结构达到新的平衡（即原子受力为零）时**的状态。这意味着，在保持晶胞形状和体积不变的前提下，我们需要让原子沿着受力方向移动，直到体系能量达到该约束下的局部最小值。这个过程被称为**离子弛豫 (Ionic Relaxation)**。

    因此，正确的做法是，对于每一个变形后的构型，都进行一次**固定晶胞、只允许原子移动**的能量最小化计算。计算收敛后得到的应力，才是与该宏观应变相对应的、物理意义正确的应力。

* **实践步骤**
    1.  在 24 个 `task.XXX` 文件夹中分别运行 `abacus`。
    2.  `INPUT` 文件中的 `calculation = 'relax'` 指示 ABACUS 进行能量最小化。由于 `STRU` 文件中的晶格矢量是固定的，程序默认不会改变晶胞，从而完美地实现了“固定晶胞、弛豫原子”这一理论要求。

##### **步骤 4：线性拟合求解 (理论推导与数据分析)**

* **理论推导**
    经过步骤3，我们为每种应变模式都获得了一系列数据点。以模式1为例，我们有4组数据点 $(\epsilon_{1}^{(k)}, \sigma_{j}^{(k)})$，其中 $k=1,2,3,4$。

    我们的目标是求解方程 $\sigma_j = C_{j1}\epsilon_1$ 中的斜率 $C_{j1}$。由于数值误差的存在，这些点不会完美地落在一条直线上。因此，我们需要使用**线性回归**（具体为**最小二乘法**）来找到最佳拟合直线。

    最小二乘法旨在找到一个斜率 $C_{j1}$，使得所有测量点与拟合直线之间的误差平方和最小：
    $$ \text{Minimize} \sum_{k=1}^{4} \left( \sigma_j^{(k)} - C_{j1}\epsilon_1^{(k)} \right)^2 $$
    该优化问题的解析解为：
    $$ C_{j1} = \frac{\sum_{k} \sigma_j^{(k)}\epsilon_1^{(k)}}{\sum_{k} (\epsilon_1^{(k)})^2} $$
    通过这个公式，我们可以从带有噪音的数据中稳健地提取出最可信的斜率值，即弹性常数。

* **实践步骤**
    1.  运行脚本将上述最小二乘法拟合过程自动化。它会读取所有任务文件夹中的应变和应力数据，对 6 种模式进行拟合，计算出所有 $C_{ij}$ 分量，并将最终的 6x6 矩阵输出。
    2.  最后，脚本还会利用得到的 $C_{ij}$ 矩阵，通过标准的 Voigt-Reuss-Hill 平均公式，计算出杨氏模量、剪切模量等工程参数。

#### 3.1.5. 总结与要点

1.  **物理基础**: 弹性常数是胡克定律在线弹性范围内的比例系数，表征材料的刚度。
2.  **数学工具**: 利用 Voigt 表示法，将复杂的四阶张量 $C_{ijkl}$ 简化为易于处理的 6x6 对称矩阵 $C_{ij}$。
3.  **计算核心**: 通过**应力-应变法**，对完美平衡结构施加一系列微小应变，计算其应力响应，然后通过线性拟合求解 $C_{ij}$。
4.  **ABACUS 流程**:
    * **Relax** 获得基态结构。
    * **Apply Strains**: 自动产生 6 种模式 x 4 种大小 = 24 个变形构型。
    * **Calculate Stresses**: 对 24 个构型进行固定晶胞的原子弛豫，得到应力。
    * **Linear Fit**: 对数据进行线性回归，从斜率中提取出所有 $C_{ij}$。
5.  **应用价值**: 计算出的弹性常数可用于检验材料的**力学稳定性** (Born-Huang 判据)，并可换算为杨氏模量、剪切模量、体积模量等工程参数，是材料计算设计中的重要环节。

### 3.2 宏观弹性模量与各向异性参数解析

#### 3.2.1 体积模量 (Bulk Modulus, B or K)

**体积模量**衡量的是材料**抵抗均匀压缩、保持其体积不变**的能力。它反映了原子间结合力的强度。

* **物理意义**：想象一下将一块材料浸入深海，水从四面八方均匀地施加压力。体积模量就描述了在这种均匀压力下，材料体积收缩的难易程度。
* **参考范围**：
    * **硬质材料 (如陶瓷、金刚石)**: 通常具有非常高的体积模量，典型值在 **200 GPa 以上**，金刚石可达 440 GPa。
    * **金属 (如钢、铝)**: 体积模量较高，通常在 **70 - 200 GPa** 的范围内。
    * **软材料 (如聚合物)**: 体积模量很低，通常**低于 10 GPa**。

#### 3.2.2 剪切模量 (Shear Modulus, G)

**剪切模量**也称为刚度模量 (Rigidity Modulus)，衡量的是材料**抵抗形状改变、保持其形状不变**的能力。

* **物理意义**：想象一下将一本厚书或一副扑克牌平放在桌上，手掌按住封面水平推动。书会发生倾斜的形变，但其体积基本不变。剪切模量就是描述这种抵抗“剪切”或“扭转”形变能力的物理量。
* **参考范围**：
    * **硬质材料 (如陶瓷、硬质合金)**: 具有很高的剪切模量，通常在 **100 - 200 GPa** 范围。
    * **普通金属**: 剪切模量较高，通常在 **25 - 80 GPa** 范围，例如钢约为 80 GPa。
    * **软材料**: 剪切模量非常低，通常远**低于 5 GPa**。

#### 3.2.3 Voigt, Reuss, Voigt-Reuss-Hill (VRH) 平均

第一性原理计算出的 $C_{ij}$ 是针对**完美单晶**的，其性质具有方向性。然而，我们日常接触的材料大多是**多晶体**。Voigt、Reuss 和 Hill 提出了三种从单晶数据估算多晶体宏观性质的近似模型。

* **Voigt (福伊特) 平均**: 假设所有晶粒的**应变 (Strain) 均一**。这通常会**高估**真实的弹性模量，给出一个**理论上限**。
* **Reuss (罗伊斯) 平均**: 假设所有晶粒的**应力 (Stress) 均一**。这通常会**低估**真实的弹性模量，给出一个**理论下限**。
* **Voigt-Reuss-Hill (VRH) 平均**: 这是最常用、最被广泛接受的近似，它直接取 Voigt 上限和 Reuss 下限的**算术平均值**，被认为是理论上最接近实际情况的值。
    > $B_{VRH} = (B_{Voigt} + B_{Reuss}) / 2$
    > $G_{VRH} = (G_{Voigt} + G_{Reuss}) / 2$

#### 3.2.4 泊松比 (Poisson's Ratio, ν)

**泊松比**描述了材料在单向拉伸或压缩时，其**横向形变与纵向形变之比**。它反映了材料在拉伸时横向收缩的趋势。

* **物理意义**：当你拉伸一根橡皮筋时，它不仅会变长（纵向形变），同时也会变细（横向形变）。泊松比就是 “变细的程度” / “变长的程度” 的比值。
* **参考范围**：
    * **橡胶类材料**: 接近 **0.5**，表示形变时体积几乎不变。
    * **大多数金属**: 在 **0.25 - 0.35** 之间，例如钢约为 0.3。
    * **陶瓷等脆性材料**: 泊松比较低，通常在 **0.1 - 0.25** 之间。
    * **特殊材料 (拉胀材料)**: 泊松比可以为负值。

#### 3.2.5 普适各向异性指数 (Universal Anisotropy Index, $A^U$)

这是一个综合性的指标，用**一个单独的数字**来量化晶体**弹性的各向异性程度**。

* **物理意义**：它衡量了材料的弹性性质（如杨氏模量）是否随方向变化，以及变化的剧烈程度。
* **解读**:
    * **$A^U$ = 0**: 表示材料是**完全各向同性 (Isotropic)** 的，其弹性在所有方向上都完全相同。
    * **$A^U$ > 0**: 表示材料是**各向异性 (Anisotropic)** 的。这个数值**越大**，说明其弹性性质的方向依赖性越强，材料在不同方向上的“硬度”差异越大。常见金属的 $A^U$ 值可以从接近0到几甚至十几。

#### 3.2.6 杨氏模量 (Young's Modulus, E)

**杨氏模量**可能是最广为人知的力学参数，它衡量的是材料在**单向拉伸或压缩**时抵抗弹性形变的能力，常被直接称为“弹性模量”或“刚度”。

* **物理意义**：想象一下拉伸一根金属丝，杨氏模量描述了需要多大的力才能使其伸长一定的长度。它是应力-应变曲线初始线性阶段的斜率。
* **与其他模量的关系**：它不是一个独立的量，可以由体积模量(B)和剪切模量(G)计算得出：$E = \frac{9BG}{3B+G}$。
* **参考范围**：
    * **高刚度材料 (如陶瓷, 钨)**: 杨氏模量非常高，通常 > 200 GPa。
    * **金属 (如钢, 钛)**: 杨氏模量较高，范围多在 100 - 200 GPa。
    * **聚合物 (如塑料)**: 杨氏模量很低，通常 < 10 GPa。

#### 3.2.7 普氏比 (Pugh's Ratio, B/G)

**普氏比**即体积模量与剪切模量之比，它是一个被广泛应用的、判断材料**韧性 (Ductility) 与脆性 (Brittleness)** 的经验判据。

* **物理意义**：它比较了材料抵抗体积改变和抵抗形状改变的能力。一个易于改变形状（低G）但难以被压缩（高B）的材料，倾向于通过塑性形变来耗散能量，表现为韧性。
* **经验判据**：
    * **B/G > 1.75**: 材料倾向于表现为**韧性**。原子倾向于通过滑移等方式重新排列（塑性形变），而不是直接断裂。大多数金属都属于这一类。
    * **B/G < 1.75**: 材料倾向于表现为**脆性**。当受力时，材料更倾向于键的断裂而不是形状的改变。大多数陶瓷属于这一类。
    * 这个 **1.75** 的临界值是由 S. F. Pugh 在1954年凭经验提出的。

#### 3.2.8 拉梅第一参数 (Lamé's First Parameter, λ)

**拉梅参数**是线性弹性理论中的两个基本常数，通常记为 $\lambda$ 和 $\mu$。我们已经熟悉的**剪切模量 G 就是拉梅第二参数 $\mu$**。

* **物理意义**：与杨氏模量或剪切模量相比，$\lambda$ 没有一个特别直观的、可以单独拎出来的物理场景。它更多地是作为一个数学参数出现在弹性力学的本构方程中，用来关联应力与应变。
* **与其他模量的关系**：它可以由体积模量和剪切模量导出：$\lambda = B - \frac{2}{3}G$。在描述弹性行为时，使用 $(\lambda, G)$ 组合与使用 $(B, G)$ 或 $(E, \nu)$ 组合是等价的。

#### 3.2.9 维氏硬度 (Vickers Hardness, Hv) 的经验估算

**硬度**衡量的是材料抵抗**局部塑性形变**（如刮擦或压痕）的能力。它不是一个纯粹的弹性参数，但可以通过弹性模量进行经验性的估算。

* **物理意义**：它反映了材料表面的耐磨损和抗压痕能力。
* **经验估算模型**：有多种基于 B 和 G 估算 $H_v$ 的模型，例如 Chen 等人提出的一个常用模型：
    > $H_v (\text{GPa}) \approx 2(k^2G)^{0.585} - 3$, 其中 $k = G/B$。
* **重要提示**：必须强调，这只是一个**经验公式**，其结果是一个理论预测值，用于趋势分析和高通量筛选，可能与实验测量的精确值存在差异。
* **参考范围**：
    * **软金属 (如铝)**: $H_v$ 通常在 1 GPa 以下。
    * **硬质合金与钢**: $H_v$ 通常在 5 - 20 GPa。
    * **超硬陶瓷 (如金刚石, c-BN)**: $H_v$ 可高达 40 - 100 GPa。

### 3.3 Calculation

#### 3.3.1 生成应变构型

- 首先进行 Cell-relax

In [ ]:
! cd cell-relax && cp INPUT_pw INPUT && abacus

Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32
                                                                                     
                              ABACUS v3.10.0

               Atomic-orbital Based Ab-initio Computation at UStc                    

                     Website: http://abacus.ustc.edu.cn/                             
               Documentation: https://abacus.deepmodeling.com/                       
                  Repository: https://github.com/abacusmodeling/abacus-develop       
                              https://github.com/deepmodeling/abacus-develop         
                      Commit: 09195430e (Fri Jun 13 14:18:31 2025 +0800)

 Tue Jun 17 18:39:14 2025
 MAKE THE DIR         : OUT.ABACUS/
 RUNNING WITH DEVICE  : GPU / NVIDIA GeForce RTX 4080

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
 Pseudopotentials with additional elect

- 使用脚本生成应变构型

In [3]:
from elastic_calc import ElasticCalculator

# cp from cell-relax to elastic-calc
elastic_calc = ElasticCalculator()
elastic_calc.gen_dfm()

gen with norm [-0.01, -0.005, 0.005, 0.01]
gen with shear [-0.01, -0.005, 0.005, 0.01]


/home/outis/miniconda3/envs/dpmd_conda/lib/python3.12/site-packages/pymatgen/core/structure.py:3111: BadPoscarWarning: Ignoring selective dynamics tag, as no ionic degrees of freedom were fixed.
  struct = Poscar.from_str(input_string, default_names=None, read_velocities=False, **kwargs).structure


In [4]:
! cd elastic-calc && chmod 755 ./elastic_run_relax.sh && ./elastic_run_relax.sh

/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.000
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32
/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.001
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32
/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.002
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32
/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.003
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32
/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.004
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32


IOStream.flush timed out
IOStream.flush timed out


/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.005
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32
/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.006
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32
/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.007
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32
/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.008
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32
/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.009
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32
/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.010
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread n

IOStream.flush timed out


/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.011
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32
/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.012
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32


...

IOStream.flush timed out
IOStream.flush timed out


/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.020
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32


IOStream.flush timed out
IOStream.flush timed out


/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.021
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32
/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.022
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32
/home/outis/Research/deepmd_learn/5_Ti2AlC_Label/elastic-calc/task.023
Info: Local MPI proc number: 1,OpenMP thread number: 1,Total thread number: 1,Local thread limit: 32


IOStream.flush timed out


#### 3.3.2 使用 pymatgen 拟合弹性常数

In [1]:
from elastic_calc import ElasticCalculator

elastic_calc = ElasticCalculator()
elastic_calc.compute_elastic()

# Elastic Constants in GPa
-304.06  -58.13  -56.88    0.00    0.00    0.00 
 -56.19 -306.34  -56.61    0.00    0.00    0.00 
 -59.04  -59.04 -269.79    0.00    0.00    0.00 
  -0.13   -0.01   -0.11 -111.55    0.00    0.00 
  -0.24   -0.25   -0.53    0.00 -111.01    0.00 
  -0.25    0.10   -0.21    0.00    0.00 -121.10 
# Bulk   Modulus BV = -136.23 GPa
# Shear  Modulus GV = -115.97 GPa
# Youngs Modulus EV = -271.01 GPa
# Poission Ratio uV = 0.17 


#### 3.3.3 对比

- 理论计算与实验数据对比

[1] M. Radovic et al., “On the elastic properties and mechanical damping of Ti3SiC2, Ti3GeC2, Ti3Si0.5Al0.5C2 and Ti2AlC in the 300–1573K temperature range,” Acta Materialia, vol. 54, no. 10, pp. 2757–2767, Jun. 2006, doi: 10.1016/j.actamat.2006.02.019.

| 物理量 | 计算结果 (DFT) | 文献实验值 (RUS) | 相对误差 |
| :--- | :--- | :--- | :--- |
| **体积模量 (Bulk Modulus, GPa)** | 138.91 | 139.6 ± 0.3 | -0.5% |
| **剪切模量 (Shear Modulus, GPa)** | 116.44 | 118.8 ± 0.3 | -2.0% |
| **杨氏模量 (Young's Modulus, GPa)** | 273.03 | 277.6 ± 0.7 | -1.6% |
| **泊松比 (Poisson's Ratio)** | 0.17 | 0.169 ± 0.0004 | +0.6% |